### Quick Look

In [1]:
import os
baudrillard = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulacra-and-simulation.txt'
baudrillard_2 = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulations.txt'
def load_data(path: str) -> str:

    text_file = os.path.join(path)
    with open(text_file, "r") as f:
        data = f.read()

    return data

_data = load_data(baudrillard)
_data_2 = load_data(baudrillard_2)
main_data = _data + _data_2

In [2]:
import numpy as np

view_line_range = (100, 150)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in main_data.split()})))

lines = main_data.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(main_data.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 14440
Number of lines: 11204
Average number of words in each line: 7.909139593002499

The lines 100 to 150:
whose operation is nuclear and genetic, no longer at all specular or discursive. It is all of 
metaphysics that is lost. No more mirror of being and appearances, of the real and its 
concept. No more imaginary coextensivity: it is genetic miniaturization that is the 
dimension of simulation. The real is produced from miniaturized cells, matrices, and 
memory banks, models of control - and it can be reproduced an indefinite number of 
times from these. It no longer needs to be rational, because it no longer measures itself 
against either an ideal or negative instance. It is no longer anything but operational. In 
fact, it is no longer really the real, because no imaginary envelops it anymore. It is a 
hyperreal, produced from a radiating synthesis of combinatory models in a hyperspace 
without atmosphere. 

By crossing into a spac

### More Developed Dive

In [2]:
from utils import data

# NOTE: paths are all specific amd hard coded. be explicit

# in the case of having multiple txt documents
baudrillard = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulacra-and-simulation.txt'
baudrillard_2 = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulations.txt'
text = data.convertManyToOne([baudrillard, baudrillard_2])
# otherwise use data.load_data(text_path) for single txt document

# set tmp location for runtime generated single txt doc combing all txt documents
tmp_path = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/tmp'
tmp_file = 'tmp_combined.txt'
# check and create if needed
if not os.path.exists(tmp_path):
    os.mkdir(tmp_path)
# save
with open(f'{tmp_path}/{tmp_file}', 'w') as f:
    f.write(text)
    f.close()


# get text title
text_tile = 'baudrillad_combined'
# check and create if needed
if not os.path.exists(f'processedData/{text_title}_preprocess.p'):

    data.preprocess_and_save(
        text_path=f'{tmp_path}/{tmp_file}',
        text_title=text_title
    )
# load nlp data for modeling
int_text, w2i, i2w, token_dict = data.load_preprocess(text_title)

CMD = f'rm -r {tmp_path}'
os.system(CMD)

In [3]:
import torch

train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CPU')

CPU


### Data Pipeline

In [4]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

def batch_data(
    words: list, 
    sequence_length: int, 
    batch_size: int) -> DataLoader:

    words = np.array(words)
    total_batch_size = batch_size * sequence_length
    n_batches = len(words) // batch_size

    words = words[:n_batches * total_batch_size]
    feature, target = [], []

    target_words_length = words[:-sequence_length]
    for idx in range(0, len(target_words_length)):

        feature.append(words[idx: idx + sequence_length])
        target.append(words[idx + sequence_length])

    batch_nums = len(words) // batch_size
    feature = feature[:batch_nums * batch_size]
    target = target[:batch_nums * batch_size]

    feature_tensors = torch.from_numpy(np.asarray(feature))
    target_tensors = torch.from_numpy(np.asarray(target))

    data_set = TensorDataset(feature_tensors, target_tensors)
    data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=True)

    return data_loader

In [16]:
# test dataloader

# test_text = range(50)
# t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

# data_iter = iter(t_loader)
# sample_x, sample_y = data_iter.next()

# print(sample_x.shape) # batch_size x sequence_length
# print(sample_x) # batch
# print()
# print(sample_y.shape) # text | len(words)
# print(sample_y) # text | list(int)

In [5]:
n_loader = batch_data(int_text, sequence_length=7, batch_size=32)
data_iter = iter(n_loader)
x,y = data_iter.next()

print(x)
print(y)

tensor([[  343,     7,     0,    61,   478,     2, 11576],
        [   50,     8,     2,    28,  2421,     4, 11651],
        [ 3323,    11,  5320,     8,  1134,  2133,    55],
        [  455,     1,  8233,  8234,     2,  8235,     5],
        [  734,     1,   150,    18,    93,   234,    22],
        [   50,     8,     2, 12161,     5,  2427,     9],
        [ 9320,     4,   512,     5,     0,   861,  1502],
        [15386, 15387,   462,    16,  2602,   132,     0],
        [ 7719,   949,     1,   363,     1,   253,    53],
        [    7,     3,    40,   103,    38,   111,   315],
        [  895,     1,  6387,     0,  6388,     7,     2],
        [    1,     0,   929,    66,     4,     1,     0],
        [    3,   173,   130,     7,  2628,   132,    25],
        [ 6303,     1,     0,  6304,   881,    84,    88],
        [   55,    39,     4,   419,  1971,    17,  7755],
        [    2,     0,  9636,  9637,    23,    37,   118],
        [    0,  2307,     4,  9210,  1801,     1,     0

### Modeling

In [6]:
from LSTM import LSTM

In [7]:
import torch
import torch.nn as nn

def forward_back_prop(
    model, 
    optimizer, 
    criterion, 
    inp, 
    target, 
    hidden,
    train_on_gpu=False):

    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()

    hidden = tuple([each.data for each in hidden])

    model.zero_grad()
    output, hidden = model(inp, hidden)

    loss = criterion(output, target)
    loss.backward()

    clip = 5
    nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()

    return loss.item(), hidden
    

    

In [8]:
def train_LSTM(
    model,
    batch_size,
    optimizer,
    criterion,
    n_epochs,
    train_loader,
    show_every_n_batches=100):

    batch_losses = []

    model.train()

    print(f' | Training for {n_epochs} epoch(s).....')
    for e in range(1, n_epochs + 1):

        hidden = model.init_hidden(batch_size)

        for batch_idx, (inputs, labels) in enumerate(train_loader, 1):

            n_batches = len(train_loader.dataset)//batch_size

            if batch_idx > n_batches:
                break

            loss, hidden = forward_back_prop(
                                model=model,
                                optimizer=optimizer,
                                criterion=criterion,
                                inp=inputs,
                                target=labels,
                                hidden=hidden
            )
            batch_losses.append(loss)

            if batch_idx % show_every_n_batches == 0:
                print(f' | EPOCH: {e:>4}/{n_epochs:>4} -- LOSS: {np.average(batch_losses)}')
                batch_losses = []

    return model



In [13]:
num_epochs = 4
learning_rate = 0.0001
vocab_size = len(w2i)
output_size = vocab_size
embedding_dim = 400
hidden_dim = 512
n_layers = 3
sequence_length = 200
batch_size = 32
show_every_n_batches = 500

model = LSTM(
        vocab_size=vocab_size,
        output_size=output_size,
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        n_layers=n_layers
)

if train_on_gpu:
    model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


In [14]:
trained_model = train_LSTM(
                    model=model,
                    batch_size=batch_size,
                    optimizer=optimizer,
                    criterion=criterion,
                    n_epochs=num_epochs,
                    train_loader=n_loader,
                    show_every_n_batches=show_every_n_batches
)

 | Training for 4 epoch(s).....
 | EPOCH:    1/   4 -- LOSS: 9.327696741104125
 | EPOCH:    1/   4 -- LOSS: 9.21197619819641
 | EPOCH:    1/   4 -- LOSS: 9.176715986251832
 | EPOCH:    1/   4 -- LOSS: 9.170046142578125
 | EPOCH:    1/   4 -- LOSS: 9.153282485961913
 | EPOCH:    2/   4 -- LOSS: 9.145766514950518
 | EPOCH:    2/   4 -- LOSS: 9.13773410987854
 | EPOCH:    2/   4 -- LOSS: 9.130745916366577
 | EPOCH:    2/   4 -- LOSS: 9.132054363250733
 | EPOCH:    2/   4 -- LOSS: 9.125159162521362
 | EPOCH:    3/   4 -- LOSS: 9.123021045183794
 | EPOCH:    3/   4 -- LOSS: 9.127365449905396
 | EPOCH:    3/   4 -- LOSS: 9.118731203079223
 | EPOCH:    3/   4 -- LOSS: 9.118500480651855
 | EPOCH:    3/   4 -- LOSS: 9.124070850372314
 | EPOCH:    4/   4 -- LOSS: 9.120307922363281
 | EPOCH:    4/   4 -- LOSS: 9.11461480331421
 | EPOCH:    4/   4 -- LOSS: 9.111514724731446
 | EPOCH:    4/   4 -- LOSS: 9.115749227523803
 | EPOCH:    4/   4 -- LOSS: 9.109112625122071


In [17]:
data.save_model(filename='models/baudrillard_v1.pt', decoder=trained_model)
# data.load_model(filename)

### Generate Text

In [25]:
import torch.nn.functional as F

def generate(
    model,
    prime_id,
    i2w,
    token_dict,
    pad_value,
    predict_len=100
):

    model.eval()

    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [i2w[prime_id]]

    for _ in range(predict_len):

        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)

        hidden = model.init_hidden(current_seq.size(0))

        output, _ = model(current_seq, hidden)

        p = F.softmax(output, dim=1).data

        if train_on_gpu:
            p = p.cpu()

        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()

        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())

        word = i2w[word_i]
        predicted.append(word)

        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i

    gen_sentences = ' '.join(predicted)

    for k, v in token_dict.items():

        ending = ' ' if k in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + v.lower(), k)

    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')

    return gen_sentences


### Generate Process

In [26]:
gen_length = 400
prime_word = 'triumph'

pad_word = data.SPECIAL_WORDS['PADDING']
generated_text = generate(

                    model=trained_model,
                    prime_id=w2i[prime_word],
                    i2w=i2w,
                    token_dict=token_dict,
                    pad_value=w2i[pad_word],
                    predict_len=gen_length
)

print(generated_text)


triumph of is is the is a of to to is is a the to is a to to is is to is to to is the the the the a of a a the the is a is to the is of is is of is a of of a a a of the to the the a of the a of to the of of the the the of of of a is is of the is a of the the is to is is of the a to the of the a of to the the of a to is of the to to a to of the the of to the to a to the to of is is of the the of is to is of the the the a of of is the to of of the to is the of a is the is to of to a is is the to a to a a of a a a is to to a the the the the the of the the a is a to the the a to to to the of the is a to the is a is the the the to is a the of of to is to of is to is a of to to to of of of to is a of a of to a the of a to the to of of the a of a to the a the is is a a to to of is to is of to of to of of of is the the is the a is of to the to to to is to the of a is is of to a the the a is of to the is the of is of to of to of a of of the is to is the the a of to is the the the a of a of is i

In [28]:
with open('baudrilliardv1_generated_text.txt', 'w') as handle:

    handle.write(generated_text)
    handle.close()